# Redes profundas

![](http://cs231n.github.io/assets/nn1/neural_net2.jpeg)

# Convoluciones

![](http://cs231n.github.io/assets/cnn/cnn.jpeg)


![](http://cs231n.github.io/assets/cnn/depthcol.jpeg)


Se mueve (convolve operation): 
[link funcionando](http://cs231n.github.io/convolutional-networks/)



# Pooling

![](http://cs231n.github.io/assets/cnn/maxpool.jpeg)



# Hacer sus propias redes

Usaremos [Keras](https://keras.io/).

In [0]:
#Definicion de librerias con la funciones que seran utilizadas por Keras. 
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten, GlobalAveragePooling2D
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
import pickle
from keras.callbacks import ModelCheckpoint
from IPython.display import Image, display
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

## Red profunda simple

Creamos un modelo del tipo Sequential.

In [4]:
model = Sequential() 
model.add(Dense(1, input_dim=100, activation="sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 101       
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [5]:
model(keras.backend.random_normal((1, 100)))

<tf.Tensor 'sequential_2/dense_3/Sigmoid:0' shape=(1, 1) dtype=float32>

**¿Podemos convertirla en una red profunda?**


In [6]:
model = Sequential() 
model.add(Dense(100, input_dim=100, activation="sigmoid"))
model.add(Dense(1, input_dim=100, activation="sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 10,201
Trainable params: 10,201
Non-trainable params: 0
_________________________________________________________________


**¿Y podemos convertirla en clasificación multiclase?**

In [7]:
model = Sequential() 
model.add(Dense(100, input_dim=100, activation="sigmoid"))
model.add(Dense(10, input_dim=100, activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1010      
Total params: 11,110
Trainable params: 11,110
Non-trainable params: 0
_________________________________________________________________


##  VGG19


![](https://cdn-images-1.medium.com/max/1600/1*cufAO77aeSWdShs3ba5ndg.jpeg)

In [0]:
modelVGG19 = Sequential() 

In [0]:
# definir primer bloque de VGG19.
modelVGG19.add(Conv2D(64, (3, 3), input_shape=(224, 224, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(64, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(MaxPooling2D((2,2), strides=(2,2)))

In [10]:
# definir segundo bloque
modelVGG19.add(Conv2D(128, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(128, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(MaxPooling2D((2,2), strides=(2,2)))
modelVGG19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
Total params: 260,160
Trainable params: 260,160
Non-trainable params: 0
_________________________________________________________________


In [11]:
# probamos modelo con "dummy" input
modelVGG19(keras.backend.random_normal((1, 224, 224, 3))).shape

TensorShape([Dimension(1), Dimension(56), Dimension(56), Dimension(128)])

En las lineas anteriores vemos como se definen los dos primeros bloques de VGG19 y lugo probamos la arquitectura usando un tensor que representa un batch de una imagen de las dimensiones pedidas.
El output de la operacion de aplicar el modelo sobre el tensor de prueba nos devuelve una salida de (*batch_size*, 56, 56, 128) que tiene sentido considerando que corresponde con lo predicho por `model.summary()`. 
El número de canales corresponde a el numero de filtros convolucionales aplicados en la conv2d_16.

Podemos ver que hasta ahora van 260160 parametros entrenables.


In [12]:
# definir tercer bloque
modelVGG19.add(Conv2D(256, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(256, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(256, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(256, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(MaxPooling2D((2,2), strides=(2,2)))
modelVGG19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
__________

In [13]:
modelVGG19.output_shape

(None, 28, 28, 256)

Podemos ver que para el tercer bloque agregamos las 4 capas convolucionales (de 256 filtros cada una) seguidas por maxpooling. Se sigue usando relu.

Finalmente, podemos ver de output_shape que las dimensiones cambiaron y ahora la salida es de  (*batch_size*, 56, 56, 256). 

In [0]:
# definir cuarto bloque
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(MaxPooling2D((2,2), strides=(2,2)))

In [15]:
# definir quinto bloque
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(Conv2D(512, (3, 3), strides=(1, 1), activation="relu", padding="same")) 
modelVGG19.add(MaxPooling2D((2,2), strides=(2,2)))
modelVGG19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
__________

Observamos que vamos en ~20 millones de parametros entrenables (20,024,384 para que no me bajen la nota).
La red tiene las 16 capas convolucionales que corresponden según el paper original y observamos que las dimensiones de salida  (*batch_size*, 7, 7, 512) corresponden a las dimensiones que debe tener. Con las 4 nuevas capas de 512 filtros de 3x3 cada una del 4 bloque, y otras tantas por el quinto bloque.
Verifico esto con el código pytorch en el que me basé [](https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py) donde puede verse que el clasificador final recibe entradas de 512 * 7 * 7.


## Agregamos capas fully connected

In [0]:
# aplanar salida para convertirla en vector (necesario para capas fully connected)
modelVGG19.add(Flatten())

In [17]:
modelVGG19.output_shape[1] == (7 * 7 * 512)

True

Como esperabamos obtenemos que despues del flatten las dimensiones son las mismas pero la forma del tensor es la que cambia. En memoria esto se ve como un cambio en la abstracción que contiene al tensor y no en el tensor mismo como podemos ver en la documentacion oficial [aca](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten).

In [0]:
modelVGG19.add(Dense(4096, activation="relu"))

In [19]:
modelVGG19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
__________

Vamos en 17 capas con parametros aprendidos. Observamos que sólo la última capa aporta 102,764,544 parametros de los 122,788,928 totales (aporta la mayoría). 
Como era de esperar, la salida tiene tantas dimensiones como neuronas definimos para la capa. 
El numero de parametros corresponde a las `(dimensiones de entrada * las dimensiones de salida) + las dimensiones de salida `.
Esto se entiende cuando se considera el bias.

In [0]:
modelVGG19.add(Dense(4096, activation="relu"))
modelVGG19.add(Dense(1000, activation="softmax"))

In [21]:
modelVGG19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
__________

Con estas ultimas dos capas la red queda con 19 capas aprendidas (no cuentan las de pooling o activaciones).
La ultima capa hace una clasificación a las mil clases que existen en el set de datos [ImageNet](http://www.image-net.org/)